<a href="https://colab.research.google.com/github/s1f10220294/Synthetic-Opinion-Dataset-Framework/blob/main/opinion_generation_tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --- ライブラリのインストール ---
!pip install gradio openai pandas ddgs sentence-transformers numpy torch requests beautifulsoup4 readability-lxml anthropic
!pip install --upgrade google-generativeai

import gradio as gr
import openai
import google.generativeai as genai
import anthropic
import pandas as pd
import random
import datetime
import os
import re # ★ 正規表現によるAI出力の解析
from google.colab import userdata
import time # ◀◀◀【変更点 1/5】レート制限（スリープ）のために追加
from google.colab import drive # ◀◀◀【変更点 2/5】Google Drive連携のために追加

# RAG / MMR
from ddgs.ddgs import DDGS
from sentence_transformers import SentenceTransformer, util
import numpy as np
import torch

# スクレイピング
import requests
from bs4 import BeautifulSoup
from readability import Document # readability をインポート

# --- 定数定義 ---

# ペルソナ属性
PERSONA_ATTRIBUTES = {
    "gender": ["男性", "女性", "その他"],
    "age": ["10代", "20代", "30代", "40代", "50代", "60代", "70代", "80代", "90代"],
    "personality": [
        "楽観的", "悲観的", "慎重", "感情的", "論理的", "直感的",
        "現実的", "理想主義的", "内向的", "外向的", "協調的", "独立的"
    ],
    "tone": [
        "丁寧", "断定的", "口語的", "攻撃的", "冷静", "情熱的",
        "皮肉的", "分析的", "謙虚", "事務的", "友好的"
    ],
    "volume": ["50文字程度", "80文字程度","100文字程度","150文字程度",'200文字程度']
}

# RAG / スクレイピング定数 (V6のまま)
MAX_CONTEXT_CHARACTERS = 4000 #コンテキストの最大文字数
CHUNK_SIZE = 600 #分割サイズ
CHUNK_OVERLAP = 200 #分割時の重なり(分割時に区切れないように)
SCRAPE_TOP_N = 5   #取得する上位のページ数
SCRAPE_TIMEOUT = 5  #タイムアウト時間(読み込みが長すぎた場合)

# MMRモデルのグローバルロード
try:
    print("MMR用のSentenceTransformerモデルをロード中...")
    mmr_model = SentenceTransformer('all-MiniLM-L6-v2')
    print("モデルのロード完了。")
except Exception as e:
    print(f"SentenceTransformerモデルのロードに失敗しました: {e}")
    mmr_model = None

# --- クライアント初期化ヘルパー ---

def get_openai_client(api_key):
    effective_api_key = api_key
    if not effective_api_key:
        print("UIにAPIキーが入力されていません。Colab Secretsから 'OPENAI_API_KEY' の読み込みを試みます...")
        try:
            effective_api_key = userdata.get('OPENAI_API_KEY')
        except Exception as e:
            print(f"Colab Secretsの読み込み失敗: {e}")
            pass
    if not effective_api_key:
         raise Exception("APIキーがUIにもColab Secretsにもありません。")

    client = openai.OpenAI(
        api_key = effective_api_key,
        # base_url = "https://api.openai.iniad.org/api/v1/",
    )
    return client

def get_gemini_client(api_key, model_name):
    """
    Geminiクライアントを初期化し、モデルオブジェクトを返す。
    セーフティ設定を緩和し、悪意のある表現の生成を試みる。
    """
    effective_api_key = api_key
    if not effective_api_key:
        print("UIにGoogle APIキーが入力されていません。Colab Secretsから 'GOOGLE_API_KEY' の読み込みを試みます...")
        try:
            effective_api_key = userdata.get('GOOGLE_API_KEY')
        except Exception as e:
            print(f"Colab Secretsの読み込み失敗: {e}")
            pass
    if not effective_api_key:
         raise Exception("Google APIキーがUIにもColab Secretsにもありません。")

    genai.configure(api_key=effective_api_key)

    # ★ 悪意のある表現（対人攻撃など）の生成を許可するため、セーフティ設定を緩和
    # これがないと、GeminiはAPIエラー（BLOCK_REASON_SAFETY）を返す可能性が高い
    safety_settings = [
        {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"},
        {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
        {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"},
        {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"},
    ]

    model = genai.GenerativeModel(
        model_name,
        safety_settings=safety_settings
    )
    return model


def get_claude_client(api_key):
    """
    Anthropic (Claude) クライアントを初期化する。
    """
    effective_api_key = api_key
    if not effective_api_key:
        print("UIにAnthropic APIキーが入力されていません。Colab Secretsから 'ANTHROPIC_API_KEY' の読み込みを試みます...")
        try:
            effective_api_key = userdata.get('ANTHROPIC_API_KEY')
        except Exception as e:
            print(f"Colab Secretsの読み込み失敗: {e}")
            pass
    if not effective_api_key:
         raise Exception("Anthropic APIキーがUIにもColab Secretsにもありません。")

    client = anthropic.Anthropic(
        api_key=effective_api_key,
    )
    return client

# --- 機能関数 (CSV) ---
# ◀◀◀【変更】CSVの保存先を「Colabローカル」と「Google Drive」の両方に変更
def save_to_csv(data_df):
    try:
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"generated_opinions_final_v7_lowest_{timestamp}.csv" # 共通のファイル名

        # --- 1. Colabローカル環境への保存 ---
        data_df.to_csv(filename, index=False, encoding='utf-8-sig')
        print(f"CSVファイル '{filename}' をColabローカルに作成しました。")

        # --- 2. Google Driveへの保存 ---
        drive_path = "/content/drive/MyDrive/"
        full_drive_path = os.path.join(drive_path, filename) # Drive用のフルパス

        # Driveのフォルダが存在するか確認（なければ作る）
        os.makedirs(drive_path, exist_ok=True)

        data_df.to_csv(full_drive_path, index=False, encoding='utf-8-sig')
        print(f"CSVファイル '{full_drive_path}' をGoogle Driveに作成しました。")

        # --- 戻り値は「ローカルのファイルパス」を返す (Gradioの File 出力コンポーネント用) ---
        return filename, f"✅ 生成とGoogle Drive/Colabローカルへの保存に成功しました。`{filename}` を確認してください。"

    except Exception as e:
        print(f"CSVファイルの保存中にエラー: {e}")
        return None, f"❌ CSVファイルの保存中にエラーが発生しました: {e}"

# --- RAG（Web検索）関数 (変更なし) ---
def perform_web_search(query, max_results=20):
    print(f"  ... 「{query}」のWeb検索を実行中 (Max {max_results}件)...")
    try:
        with DDGS() as ddgs:
            results = ddgs.text(query, max_results=max_results, region='jp-jp')
        if not results:
            print("  ... 検索結果が0件でした。")
            return []
        search_data = [
            {"title": r.get("title", "タイトルなし"),
             "snippet": r.get("body", "本文なし"),
             "url": r.get("href", "URLなし")}
            for r in results
        ]
        print(f"  ... {len(search_data)}件の検索結果（フォールバック候補）を取得しました。")
        return search_data
    except Exception as e:
        print(f"  ... Web検索中にエラーが発生しました: {e}")
        return []

# --- 【V6】readability ＋ 文字化け対策 ＋ Chunk600 (変更なし) ---
def scrape_and_chunk_pages(search_results, top_n=5, chunk_size=600, overlap=200):
    print(f"  ... 高品質スクレイピング（Top {top_n}件, readability使用, Chunk={chunk_size}）を開始します...")
    high_quality_chunks = []

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Referer': 'https://www.google.com/'
    }

    target_urls = [res['url'] for res in search_results[:top_n] if res.get('url')]
    if not target_urls:
        print("  ... スクレイピング対象のURLがありませんでした。")
        return []

    for url in target_urls:
        try:
            response = requests.get(url, headers=headers, timeout=SCRAPE_TIMEOUT)
            response.raise_for_status()
            response.encoding = response.apparent_encoding

            doc = Document(response.text)
            article_title = doc.title()
            article_html = doc.summary()

            soup = BeautifulSoup(article_html, 'html.parser')
            text = soup.get_text(separator='\n', strip=True)
            text = re.sub(r'\n{3,}', '\n\n', text)

            if not text or len(text) < chunk_size / 2:
                print(f"  ... [失敗] {url} からreadabilityで十分なテキストを抽出できません。")
                continue

            print(f"  ... [成功] {url} (Title: {article_title}) からテキスト抽出。チャンキング中...")
            start = 0
            while start < len(text):
                end = start + chunk_size
                chunk = text[start:end]
                high_quality_chunks.append({
                    "title": f"チャンク (Title: {article_title})",
                    "snippet": chunk,
                    "url": url
                })
                start += chunk_size - overlap
                if start + overlap >= len(text):
                    break

        except requests.RequestException as e:
            print(f"  ... [失敗] {url} のダウンロードエラー: {e}")
        except Exception as e:
            print(f"  ... [失敗] {url} の解析エラー (readability等): {e}")

    print(f"  ... スクレイピング完了。合計 {len(high_quality_chunks)} 件の高品質チャンクを生成しました。")
    return high_quality_chunks

# --- MMR（多様性リランキング）関数 (変更なし) ---
def apply_mmr(query, search_results, top_k=5, lambda_val=0.5):
    valid_results = [res for res in search_results if res.get('snippet') and res.get('snippet').strip()]
    if not mmr_model or not valid_results:
        print("  ... MMRモデルがロードされていないか、有効なスニペットが空です。MMRをスキップします。")
        return search_results[:top_k]
    print(f"  ... MMR（λ={lambda_val}）を適用中... {len(valid_results)}件 -> {top_k}件")
    try:
        query_embedding = mmr_model.encode(query, convert_to_tensor=True)
        doc_embeddings = mmr_model.encode([res["snippet"] for res in valid_results], convert_to_tensor=True)
        cosine_scores = util.cos_sim(query_embedding, doc_embeddings)[0]
        selected_doc_indices = []
        while len(selected_doc_indices) < min(top_k, len(valid_results)):
            remaining_doc_indices = list(set(range(len(valid_results))) - set(selected_doc_indices))
            mmr_scores = []
            for i in remaining_doc_indices:
                relevance_score = cosine_scores[i]
                if not selected_doc_indices:
                    diversity_score = 0
                else:
                    selected_embeddings = doc_embeddings[selected_doc_indices]
                    similarity_to_selected = util.cos_sim(doc_embeddings[i], selected_embeddings)
                    diversity_score = torch.max(similarity_to_selected)
                mmr_score = lambda_val * relevance_score - (1 - lambda_val) * diversity_score
                mmr_scores.append((mmr_score, i))
            if not mmr_scores:
                break
            best_mmr_score, best_index = max(mmr_scores, key=lambda x: x[0])
            selected_doc_indices.append(best_index)
        mmr_results = [valid_results[i] for i in selected_doc_indices]
        print("  ... MMRによる再ランキング完了。")
        return mmr_results
    except Exception as e:
        print(f"  ... MMRの計算中にエラーが発生しました: {e}。MMRをスキップします。")
        return valid_results[:top_k]

# --- AI関数群 ---

# --- 【API 1/3】プロフィール ＋ 1クエリ ＋ 理由 (V7) ---
def generate_profile_and_query(persona_dict, theme, stance,
                               api_key, google_api_key, claude_api_key, # ◀ 引数追加
                               model):
    print(f"    ... 【API 1/3】プロフィールと検索クエリ(＋理由)を同時生成中 (Model: {model})...")

    # (user_prompt は変更なし)
    user_prompt = f"""
    あなたはプロのプロフィール作家兼リサーチャーです。
    以下の指示に従い、3つの項目（プロフィール、検索クエリ、理由）をリアリティをもって出力してください。
    # 1. 人物属性
    - 性別: {persona_dict['gender']}
    - 年齢: {persona_dict['age']}
    - 性格: {persona_dict['personality']}
    - 口調: {persona_dict['tone']}
    # 2. タスク
    以下の3つの項目を、指定された形式で生成してください。
    ---
    プロフィール:
    属性に基づき、以下の3要素を考慮したプロフィールを100〜200文字程度で創作してください。
    1.【基本情報】: 性別、年齢、社会的役割、家族構成など
    2.【パーソナリティ】: その社会的な役割や口調を反映した、その人の価値観
    3.【生活の舞台や社会的立場】:生活環境や周囲との関係性（家族関係や交友関係など）やどんな立ち位置か
---
    検索クエリ:
  （ここに、上記であなたが創作した「プロフィール」を持つ人物が、「{theme}」について「{stance}」の立場で情報収集する際に検索しそうな**具体的な検索クエリ**を**1つだけ**生成してください）
    ---
    理由:
  （ここに、なぜその「プロフィール」の人物が、その「検索クエリ」を入力するに至ったのか、その思考プロセスや動機を簡潔に説明してください）
---
"""
    fallback_profile = f"（フォールバック: {persona_dict['age']} {persona_dict['gender']}, {persona_dict['personality']}）"
    fallback_query = f"{theme} {stance} 理由"
    fallback_reason = "フォールバック"

    try:
        full_response_text = ""

        # ▼▼▼ モデル分岐 ▼▼▼
        if model.startswith("gemini-"):
            client = get_gemini_client(google_api_key, model)
            generation_config = genai.types.GenerationConfig(temperature=1.0)
            response = client.generate_content(user_prompt, generation_config=generation_config)
            full_response_text = response.text

        elif model.startswith("gpt-"):
            client = get_openai_client(api_key)
            response = client.chat.completions.create(
                model=model,
                messages=[{"role": "user", "content": user_prompt}],
                temperature=1.0,
                max_tokens=500
            )
            full_response_text = response.choices[0].message.content.strip()

        elif model.startswith("claude-"): # ◀◀◀ Claude分岐を追加
            client = get_claude_client(claude_api_key)
            response = client.messages.create(
                model=model,
                messages=[{"role": "user", "content": user_prompt}],
                temperature=1.0,
                max_tokens=500
            )
            full_response_text = response.content[0].text

        else:
            raise Exception(f"未対応のモデルです: {model}")
        # ▲▲▲ モデル分岐 ▲▲▲

        # ★★★ 修正: 正規表現ロジックの強化 ★★★
        # 区切り線(---)があってもなくても、次の項目の見出しが出現したらそこで区切る

        # 1. プロフィール抽出
        profile_match = re.search(r"プロフィール[:：\s]*(.*?)(?=\n.*?検索クエリ[:：]|\n.*?---|---|$)", full_response_text, re.DOTALL | re.IGNORECASE)
        profile_text = profile_match.group(1).strip() if profile_match else fallback_profile

        # 2. 検索クエリ抽出
        query_match = re.search(r"検索クエリ[:：\s]*(.*?)(?=\n.*?理由[:：]|\n.*?---|---|$)", full_response_text, re.DOTALL | re.IGNORECASE)
        query_text = query_match.group(1).strip() if query_match else fallback_query

        # 3. 理由抽出
        reason_match = re.search(r"理由[:：\s]*(.*)", full_response_text, re.DOTALL | re.IGNORECASE)
        reason_text = reason_match.group(1).strip() if reason_match else fallback_reason

        print(f"    ... API 1/3 完了。")
        print(f"    ... プロフィール: {profile_text[:50]}...")
        print(f"    ... 検索クエリ: {query_text}")

        return {
            "profile": profile_text,
            "query": query_text,
            "reason": reason_text
        }
    except Exception as e:
        print(f"    ... API 1/3 (プロフィール生成) エラー: {e}")
        if "BLOCK_REASON_SAFETY" in str(e):
             print("    ... ★★★ Geminiのセーフティ機能によりブロックされました ★★★")
             return {"profile": f"（API 1/3 エラー: Geminiによりブロック）", "query": fallback_query, "reason": f"APIエラー: {e}"}
        # Claudeもエラー時にここにフォールバックする
        return {"profile": f"（API 1/3 エラー: {e}）", "query": fallback_query, "reason": f"APIエラー: {e}"}


# --- 【API 2/3】ナラティブ構築 (旧: RAG具体的事実の抽出) ---
def summarize_rag_context(rag_query, mmr_results,
                          api_key, google_api_key, claude_api_key,
                          model,
                          profile_text, # ◀ 追加: エピソード生成用
                          theme):       # ◀ 追加: エピソード生成用

    print(f"    ... 【API 2/3】ナラティブ構築（体験＋知識）を実行中 (Model: {model})...")

    # --- 1. スニペットテキストの構築 ---
    snippets_text = ""
    has_snippets = False

    if mmr_results and len(mmr_results) > 0:
        has_snippets = True
        for i, res in enumerate(mmr_results):
            snippets_text += f"--- スニペット {i+1} (出典: {res.get('url', 'N/A')}) ---\n"
            snippets_text += f"{res.get('snippet', '')}\n\n"

    # --- 2. プロンプトの分岐 (事実あり vs 事実なし) ---
    if has_snippets:
        # パターンA：検索結果あり（事実と体験を融合）
        user_prompt = f"""
あなたは優秀なノンフィクション作家です。
提供された「検索結果」から客観的事実を抽出し、それを「ペルソナ」の生活に落とし込んだ「体験」として構成してください。

# 1. シミュレーション対象（ペルソナ）
{profile_text}

# 2. 検索クエリ
{rag_query}

# 3. 参考スニペット
{snippets_text}

# 指示
以下の2つのセクションを必ず出力してください。

【知識】
スニペットから、テーマに関連する「客観的な事実・数値・議論」を簡潔に抽出して箇条書きにしてください。

【体験】
知識も考慮し、このペルソナが日常生活の中で直面した、テーマに関連した印象的な出来事を「心の揺れ」や「感覚(五感)」を踏まえて、150文字程度で詳細に描写してください。

"""
    else:
        # パターンB：検索結果なし（ペルソナからの想像のみ）
        print("    ... 要約するスニペットがありませんでした。想像モードでエピソードを作成します。")
        user_prompt = f"""
あなたは優秀な小説家です。
今回の検索では具体的な情報が得られませんでしたが、この「ペルソナ」の性格と生活環境に基づき、テーマ「{theme}」に関して彼/彼女が経験したであろう日常の一コマをシミュレートしてください。

# 1. シミュレーション対象（ペルソナ）
{profile_text}

# 2. テーマ
{theme}

# 指示
以下の2つのセクションを必ず出力してください。


【体験】
このペルソナがテーマに関して最近経験したであろう「印象的な出来事」や「心の揺れ」を、五感（音、光、匂いなど）を用いたリアリティのある文章で150文字程度で描写してください。
（事実データがないため、あくまで主観的な体験として書いてください）

【知識】
（検索結果がないため、以下のように記述してください）
・特になし（自身の生活実感のみに基づく）
"""
    try:
        summary = ""

        # ▼▼▼ モデル分岐 ▼▼▼
        if model.startswith("gemini-"):
            client = get_gemini_client(google_api_key, model)
            generation_config = genai.types.GenerationConfig(temperature=0.0)
            response = client.generate_content(user_prompt, generation_config=generation_config)
            summary = response.text

        elif model.startswith("gpt-"):
            client = get_openai_client(api_key)
            response = client.chat.completions.create(
                model=model,
                messages=[{"role": "user", "content": user_prompt}],
                temperature=0.0,
                max_tokens=500
            )
            summary = response.choices[0].message.content.strip()

        elif model.startswith("claude-"): # ◀◀◀ Claude分岐を追加
            client = get_claude_client(claude_api_key)
            response = client.messages.create(
                model=model,
                messages=[{"role": "user", "content": user_prompt}],
                temperature=0.0,
                max_tokens=500
            )
            summary = response.content[0].text

        else:
            raise Exception(f"未対応のモデルです: {model}")
        # ▲▲▲ モデル分岐 ▲▲▲

        print(f"    ... 事実抽出完了。")
        return summary

    except Exception as e:
        print(f"    ... 事実抽出APIエラー: {e}")
        if "BLOCK_REASON_SAFETY" in str(e):
             print("    ... ★★★ Geminiのセーフティ機能によりブロックされました ★★★")
             return f"（事実抽出APIエラー: Geminiによりブロック）"
        return f"（事実抽出APIエラー: {e}）"



# --- 【API 3/3】意見生成 (V7: 高スコア版シンプルプロンプト) ---
def generate_and_parse_all_opinions(
    theme,
    stance,
    profile_text,
    context_text,
    volume_text,
    malice_type,
    api_key,
    google_api_key,
    claude_api_key, # ◀ 引数追加
    model,
    chunk_count,
    logical_part,
    emotional_part):

    print(f"    ... 【API 3/3】「5つの意見と確率」をリクエスト中 (Model: {model})...")

    # (malice_instruction と user_prompt は変更なし)
    context_section = ""
    if context_text:
        context_section = f"■ 参照知識・具体的体験：\n{context_text}\n"
    malice_instruction = ""
    if malice_type in ["早まった一般化", "対人攻撃", "藁人形論法"]:
        malice_instruction = f"（追加指示：この意見には、意図的に「{malice_type}」という論理的誤謬を含めてください）"
        context_section = f"■ 参照知識・体験：\n{context_text}\n"

    user_prompt = f"""
以下の設定に基づき、この人物が抱くであろう意見の確率分布をシミュレートしてください。

■ シミュレーション対象（ペルソナ）：
{profile_text}

{context_section}

■ タスク：
テーマ「{theme}」に対し、この人物が**【{stance}】**という立場を堅持したまま、体験や知識に基づいた5つのリアリティある意見（各{volume_text}）を出力せよ。
※この人物の現在の思考・口調バランスは【論理{logical_part}割 ： 感情 {emotional_part}割】です。
　(感情優位ならオノマトペを交えて文体などを崩し、論理優位なら数字などを用いて淡々と記述してください。)
{malice_instruction}

■ 確率分布の条件：
「{stance}」の枠内で、統計的に最もありふれた意見（0.40+）から、最も出現しにくい主観的な意見（0.05）まで、グラデーションをつけてシミュレートすること。

■ シミュレーション結果の出力形式：
1. （ここに1つ目の意見） (0.40)
2. （ここに2つ目の意見） (0.30)
3. （ここに3つ目の意見） (0.15)
4. （ここに4つ目の意見） (0.10)
5. （ここに5つ目の意見） (0.05)

今、この人物ならどのような言葉でこのテーマを語るだろうか？
"""
    full_prompt = user_prompt
    full_response_text = ""

    try:
        # ▼▼▼ モデル分岐 ▼▼▼
        if model.startswith("gemini-"):
            client = get_gemini_client(google_api_key, model)
            generation_config = genai.types.GenerationConfig(temperature=1.0, top_p=1.0)
            response = client.generate_content(user_prompt, generation_config=generation_config)
            full_response_text = response.text

        elif model.startswith("gpt-"):
            client = get_openai_client(api_key)
            response = client.chat.completions.create(
                model=model,
                messages=[{"role": "user", "content": user_prompt}],
                temperature=1.0,
                top_p=1.0,
                max_tokens=1500
            )
            full_response_text = response.choices[0].message.content.strip()

        elif model.startswith("claude-"): # ◀◀◀ Claude分岐を追加
            client = get_claude_client(claude_api_key)
            response = client.messages.create(
                model=model,
                messages=[{"role": "user", "content": user_prompt}],
                temperature=1.0,
                max_tokens=1500
            )
            full_response_text = response.content[0].text

        else:
            raise Exception(f"未対応のモデルです: {model}")
        # ▲▲▲ モデル分岐 ▲▲▲

        # (確率的発想の「解析ロジック」は変更なし)
        prob_pattern = re.compile(
            r"[\(（]\s*(?:(?:確率|probability)\s*[:：]?)?\s*(\d+\.?\d*)\s*[%]?\s*[\)）]\s*$",
            re.IGNORECASE
        )
        opinion_blocks = re.split(r'^\s*(?:\d+|[①②③④⑤])[.)―]?\s+', full_response_text, flags=re.MULTILINE)
        candidates = []
        for block in opinion_blocks:
            if not block.strip():
                continue
            match = prob_pattern.search(block)
            if match:
                try:
                    prob_str = match.group(1)
                    prob_val = float(prob_str)
                    if prob_val > 1.0:
                        prob_val = prob_val / 100.0
                    opinion_text = prob_pattern.sub('', block).strip()
                    if opinion_text:
                        candidates.append((opinion_text, prob_val))
                except Exception as e:
                    print(f"    ... 確率の解析中に一部エラー: {e}")
            else:
                print(f"    ... 警告: 確率を解析できないブロックがありました。無視します。 -> '{block[:50]}...'")

        parsed_count = len(candidates)
        if parsed_count > 0:
            print(f"    ... {parsed_count}件の候補を解析。")
            return candidates, full_prompt, full_response_text
        else:
            print(f"    ... 警告: 確率の解析に失敗。レスポンス全体を1つの意見として扱します。")
            fallback_opinion = re.sub(r'^\s*(?:\d+[.)]|\*|\-|\・)\s+', '', full_response_text).strip()
            candidates = [(fallback_opinion, -1.0)]
            return candidates, full_prompt, full_response_text

    except Exception as e:
        error_message = f"APIの呼び出し中にエラーが発生しました: {e}"
        if "BLOCK_REASON_SAFETY" in str(e):
             print("    ... ★★★ Geminiのセーフティ機能によりブロックされました ★★★")
             error_message = f"API Error: Geminiによりブロックされました"
        return f"API Error: {error_message}", None, ""



# --- ★★★ メインの処理関数 (V7: 最低確率1件を保存) ★★★
def generate_opinions_only(theme, num_outputs, num_pro, num_neutral, num_con,
                            malice_choice, num_hasty, num_ad_hominem, num_straw_man,
                            unified_api_key,
                            model):

    # ◀◀◀【変更点 4/5】Google Driveをマウント（連携）する
    try:
        drive.mount('/content/drive')
        print("Google Driveのマウントに成功しました。")
    except Exception as e:
        print(f"Google Driveのマウントに失敗しました: {e}")
    # ▲▲▲ ここまで追加 ▲▲▲

    # ▼▼▼ 【追加】キーの振り分けロジック ▼▼▼
    # まず、すべてのキーを「空」に初期化
    api_key = None
    google_api_key = None
    claude_api_key = None

    # 選択されたモデル名に応じて、1つのキーを正しい変数に割り当てる
    if model.startswith("gpt-"):
        api_key = unified_api_key
        print("OpenAI APIキー（UI入力）を使用します。")
    elif model.startswith("gemini-"):
        google_api_key = unified_api_key
        print("Google AI APIキー（UI入力）を使用します。")
    elif model.startswith("claude-"):
        claude_api_key = unified_api_key
        print("Anthropic APIキー（UI入力）を使用します。")
    # ▲▲▲ 【追加】ここまで ▲▲▲

    if not theme:
        return "「テーマ」を入力してください。", None, None, None

    # ▼▼▼ APIキーの警告をモデル別に変更 ▼▼▼
    if model.startswith("gpt-") and not api_key:
        print("警告: OpenAI APIキーが入力されていません。Colab Secretsを試行します...")
    if model.startswith("gemini-") and not google_api_key:
        print("警告: Google APIキーが入力されていません。Colab Secretsを試行します...")
    if model.startswith("claude-") and not claude_api_key: # ◀ 追加
        print("警告: Anthropic APIキーが入力されていません。Colab Secretsを試行します...")
    # ▲▲▲ 変更ここまで ▲▲▲

    if not mmr_model:
        return "❌ 実行エラー: MMRモデルのロードに失敗しました。ランタイムを再起動してください。", None, None, None

    # ... (スタンスと悪意のリストを作成する部分は変更なし) ...
    num_pro = int(num_pro) if num_pro is not None else 0
    num_neutral = int(num_neutral) if num_neutral is not None else 0
    num_con = int(num_con) if num_con is not None else 0
    num_hasty = int(num_hasty) if num_hasty is not None else 0
    num_ad_hominem = int(num_ad_hominem) if num_ad_hominem is not None else 0
    num_straw_man = int(num_straw_man) if num_straw_man is not None else 0
    num_opinions_to_generate = int(num_outputs)
    stances = []
    if (num_pro + num_neutral + num_con) == num_opinions_to_generate:
        stances.extend(['賛成'] * num_pro)
        stances.extend(['中立'] * num_neutral)
        stances.extend(['反対'] * num_con)
    else:
        print(f"立場の内訳が目標意見数（{num_opinions_to_generate}）と一致しないため、ランダムに割り当てます。")
        stances = random.choices(['賛成', '中立', '反対'], k=num_opinions_to_generate)
    malice_types = []
    if malice_choice == "悪意あり":
        malice_types.extend(["早まった一般化"] * num_hasty)
        malice_types.extend(["対人攻撃"] * num_ad_hominem)
        malice_types.extend(["藁人形論法"] * num_straw_man)
        num_specified_malice = len(malice_types)
        num_remaining = num_opinions_to_generate - num_specified_malice
        if num_remaining > 0:
            malice_types.extend(["なし"] * num_remaining)
        else:
            malice_types = malice_types[:num_opinions_to_generate]
    else:
        malice_types.extend(["なし"] * num_opinions_to_generate)
    random.shuffle(stances)
    random.shuffle(malice_types)
    combined_attributes = list(zip(stances, malice_types))
    all_rows_data = []
    formatted_output = ""
    total_opinions_generated = 0
    print(f"合計 {num_opinions_to_generate} 件の意見生成（1件あたりAPI 3回＋RAG）を開始します...")

    # --- メインループ (V7) ---
    for i, (base_stance, malice_type) in enumerate(combined_attributes):
        opinion_id = i + 1

        # ◀◀◀ ここで思考比率を決定 1/3 ▶▶▶
        RATIO_OPTIONS = ["10:0", "9:1", "8:2", "7:3", "6:4", "5:5", "4:6", "3:7", "2:8", "1:9", "0:10"]
        WEIGHTS = [2, 3, 5, 9, 13, 15, 13, 9, 5, 3, 2]
        chosen_ratio = random.choices(RATIO_OPTIONS, weights=WEIGHTS, k=1)[0]
        logical_part, emotional_part = chosen_ratio.split(':')

        # ★ 実験2：API 3に渡すための「詳細な立場」をここで決定
        if base_stance == "賛成":
            detailed_stance = random.choice([
                # --- 強い推進（分布の端を形成） ---
                "全面的に賛成", "確信を持って賛成", "強く賛成",
                # --- 標準的な賛成（分布の中核） ---
                "概ね賛成", "どちらかといえば賛成", "葛藤しつつも賛成",
                # --- 境界線上の賛成（反対派と混ざる領域） ---
                "一応賛成", "しぶしぶ賛成", "やむを得ず賛成", "条件付きで賛成"
            ])
        elif base_stance == "反対":
            detailed_stance = random.choice([
                # --- 強い拒絶（反対側の端） ---
                "全面的に反対", "断固として反対", "強く反対",
                # --- 標準的な反対 ---
                "概ね反対", "どちらかといえば反対", "葛藤しつつも反対",
                # --- 境界線上の反対（賛成派と混ざる領域） ---
                "一応反対", "しぶしぶ反対", "やむを得ず反対", "条件付きで反対"
            ])
        else:
          detailed_stance = "中立"

        #print(f"--- 意見 {opinion_id}/{num_opinions_to_generate} (立場: {stances}) を生成中 ---")
        print(f"--- 意見 {opinion_id}/{num_opinions_to_generate} (立場: {base_stance}  を生成中 ---")
        random_persona = {key: random.choice(values) for key, values in PERSONA_ATTRIBUTES.items()}
        print(f"  ... 基本属性: {random_persona['age']}, {random_persona['personality']}, {random_persona['volume']}")

        # ★ ステップ1: プロフィールとクエリを同時生成 (API 1/3)
        api1_response = generate_profile_and_query(
            random_persona,
            theme,
            base_stance,
            api_key,
            google_api_key,
            claude_api_key, # ◀ 引数追加
            model=model
        )
        profile_text = api1_response["profile"]
        rag_query = api1_response["query"]
        query_reason = api1_response["reason"]

        # ★ ステップ2.5: ハイブリッドRAG
        ddgs_search_results = perform_web_search(rag_query, max_results=20)
        high_quality_chunks = scrape_and_chunk_pages(
            ddgs_search_results,
            top_n=SCRAPE_TOP_N,
            chunk_size=CHUNK_SIZE,
            overlap=CHUNK_OVERLAP
        )
        current_chunk_count = len(high_quality_chunks)
        snippets_for_mmr = []
        rag_source_type = ""
        if high_quality_chunks:
            print(f"  ... [RAG] 高品質チャンク ({len(high_quality_chunks)}件) をMMRに渡します。")
            snippets_for_mmr = high_quality_chunks
            rag_source_type = "Scraped Chunks (Readability V6)"
        else:
            print(f"  ... [RAG] スクレイピング失敗。フォールバック (ddgs要約 {len(ddgs_search_results)}件) をMMRに渡します。")
            snippets_for_mmr = ddgs_search_results
            rag_source_type = "Fallback DDGS"
        mmr_results = apply_mmr(rag_query, snippets_for_mmr, top_k=5, lambda_val=0.5)

        # ★ ステップ2.6: RAG 具体的事実抽出 (API 2/3)
        clean_extracted_facts = summarize_rag_context(
            rag_query,
            mmr_results,
            api_key,
            google_api_key,
            claude_api_key, # ◀ 引数追加
            model=model,
            profile_text=profile_text, # ★ここを追加
            theme=theme

        )
        noisy_context_for_csv = ""
        if mmr_results:
            noisy_context_for_csv += f"以下は、{rag_source_type} から抽出したスニペットです。\n"
            for j, res in enumerate(mmr_results):
                noisy_context_for_csv += f"--- 情報 {j+1} (出典: {res.get('url', 'N/A')}) ---\n{res.get('snippet', '')}\n"
        else:
            noisy_context_for_csv = "（MMRによるスニペット選定結果なし）"


        # ★ ステップ3: 確率的意見生成 (API 3/3)
        candidates_list, used_prompt, full_response = generate_and_parse_all_opinions(
            theme=theme,
            stance=detailed_stance,
            profile_text=profile_text,
            context_text=clean_extracted_facts,
            volume_text=random_persona['volume'],
            malice_type=malice_type,
            api_key=api_key,
            google_api_key=google_api_key,
            claude_api_key=claude_api_key, # ◀ 引数追加
            model=model,
            chunk_count=current_chunk_count,
            # ◀◀◀ 引数に追加 2/3 ▶▶▶
            logical_part=logical_part,
            emotional_part=emotional_part
        )

        # 4. APIエラーハンドリング
        if not isinstance(candidates_list, list):
            print(f"    ... 意見 {opinion_id} でエラー発生。スキップします。エラー: {candidates_list}")
            formatted_output += f"--- 意見 {opinion_id} (APIエラー) ---\n{candidates_list}\n\n"
            continue

        # 5. ★★★ V7: 最低確率の候補を1件だけ選定 ★★★
        valid_candidates = [c for c in candidates_list if c[1] >= 0]
        if not valid_candidates:
            print(f"    ... 意見 {opinion_id} で確率の解析に失敗。スキップします。")
            formatted_output += f"--- 意見 {opinion_id} (全候補の確率解析エラー) ---\n\n"
            continue

        try:
            # 確率が高い順（0.45 -> 0.05）に並んでいることを前提とします
            # インデックス 0, 1, 2, 3 が「標準的な意見」、4 が「最低確率(0.05)」

            if random.random() < 0.70:
                # 70%の確率で「インデックス 4 (0.05の意見)」を選択
                # 万が一、AIが4つしか出さなかった場合を考慮して min でガード
                idx = min(4, len(valid_candidates) - 1)
                selected_candidate = valid_candidates[idx]
                selection_type = "Lowest (70% route)"
            else:
                # 30%の確率で「インデックス 0〜3」からランダムに選択
                # 候補が少ない場合は全部から選ぶ
                others = valid_candidates[:4] if len(valid_candidates) > 1 else valid_candidates
                selected_candidate = random.choice(others)
                selection_type = "Random Others (30% route)"

            selected_opinion_text = selected_candidate[0]
            selected_probability = selected_candidate[1]

            print(f"    ... {len(valid_candidates)}件中 '{selection_type}' で 確率:{selected_probability:.4f} を選択。")
        except Exception as e:
            print(f"    ... 意見 {opinion_id} での意見選定中にエラー: {e}。スキップします。")
            formatted_output += f"--- 意見 {opinion_id} (意見選定エラー: {e}) ---\n\n"
            continue

        # 6. 選ばれた1件のみをCSVデータに追加
        total_opinions_generated += 1
        opinion_text_cleaned = re.sub(r'^\s*(?:\d+[.)]|\*|\-|\・)\s+', '', selected_opinion_text).strip()
        row_data = {
            "Opinion_ID": opinion_id,
            "生成された意見": opinion_text_cleaned,
            "テーマ": theme,
            "立場": base_stance,
            "詳細な立場": detailed_stance,
            "思考バランス(論理:感情)": chosen_ratio,
            "Selection_Type": selection_type,
            "悪意の有無": "なし" if malice_type == "なし" else "あり",
            "悪意の種類": malice_type if malice_type != "あり" else "指定なし",
            "性別": random_persona['gender'],
            "年齢": random_persona['age'],
            "性格": random_persona['personality'],
            "口調": random_persona['tone'],
            "文章量": random_persona['volume'],
            "Profile": profile_text,
            "RAG_Source_Type": rag_source_type,
            "RAG_Query": rag_query,
            "RAG_Query_Reason": query_reason,
            "RAG_Clean_Context": clean_extracted_facts,
            "RAG_Raw_Snippets": noisy_context_for_csv,
            "Selected_Probability": selected_probability,
            "Num_Candidates_Parsed": len(candidates_list),
            "Num_Candidates_Valid": len(valid_candidates),
            "生成に使用したプロンプト": used_prompt,
            "Full_API_Response": full_response
        }
        all_rows_data.append(row_data)

        # 7. プレビューに選ばれた1件のみを表示
        persona_str = f"属性:{random_persona['age']},{random_persona['personality']} {profile_text}"
        prob_info = f"(選択された確率:{selected_probability:.4f})"
        rag_info = f"[{rag_source_type}]"
        formatted_output += f"--- D {total_opinions_generated} (ID:{opinion_id}) {prob_info} {rag_info} ---\n[ペルソナ: {persona_str}]\n[RAGクエリ: {rag_query}]\n[RAG抽出文: {clean_extracted_facts[:100]}...]\n\n{opinion_text_cleaned}\n\n"

        # ◀◀◀【変更点 5/5】レート制限対策のスリープを追加
        if model.startswith("gemini-"):
            print(f"  ... 意見 {opinion_id} 完了。[Gemini] レート制限対策のため 2秒待機します...")
            time.sleep(2) # ◀◀◀ 1件ごとに2秒待機
        else:
            print(f"  ... 意見 {opinion_id} 完了。")
        # ▲▲▲ ここまで追加 ▲▲▲

    print(f"全 {num_opinions_to_generate} 件の生成が完了しました。")

    if not all_rows_data:
        print("データが1件も生成されませんでした。")
        return "❌ データが1件も生成されませんでした。APIキーやプロンプトを確認してください。", "", None, None
    try:
        df = pd.DataFrame(all_rows_data)
        filepath, status = save_to_csv(df)
        print(status)
        return status, formatted_output.strip(), df, filepath
    except Exception as e:
        print(f"DataFrame作成またはCSV保存エラー: {e}")
        return f"❌ DataFrameの作成またはCSVへの保存中にエラー: {e}", "", None, None


# --- Gradioインターフェースの定義 (説明文をV7に戻す) ---
with gr.Blocks() as demo:
    gr.Markdown("<h1>人工意見データセットの自動構築アプリ</h1>")

    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("<h5>▼ APIキー設定</h5>")

            # ▼▼▼ 3つの入力をこれ1つに置き換える ▼▼▼
            unified_api_key_input = gr.Textbox(
                label="APIキー",
                placeholder="sk-...",
                type="password",
                info="モデルに応じて、使用するAPIキー（OpenAI, Google, Anthropic）を1つ入力してください。"
            )
            # ▲▲▲ 置き換えここまで ▲▲▲


            # ▼▼▼ モデル選択の choices を修正 ▼▼▼
            model_input = gr.Dropdown(
                label="使用するモデル",
                choices=[
                    "gpt-4o",
                    "gpt-3.5-turbo",
                    "gpt-4-turbo",
                    "gemini-2.5-flash",
                    "gemini-2.5-pro",
                    "claude-haiku-4-5",
                    "claude-sonnet-4-5",

                ],
                value="gpt-4o",
                info="API呼び出しに使用するモデルを選択します。"
            )
            # ▲▲▲ 修正ここまで ▲▲▲

            gr.Markdown("---")
            theme_input = gr.Textbox(label="テーマ")
            num_outputs_input = gr.Number(label="最終的な意見の数 (× 3 APIコール + RAG)", value=3, step=1, minimum=1)

            gr.Markdown("<h5>▼ 立場の内訳（合計が「最終的な意見の数」と合わない場合はランダム）</h5>")
            with gr.Row():
                num_pro_input = gr.Number(label="賛成", value=1, step=1, minimum=0)
                num_con_input = gr.Number(label="反対", value=1, step=1, minimum=0)

            gr.Markdown("<h5>▼ 悪意の有無</h5>")
            malice_choice_radio = gr.Radio(["悪意なし", "悪意あり"], label="生成する意見に悪意を含めますか？", value="悪意なし")
            with gr.Group(visible=False) as malice_details_group:
                gr.Markdown("<h6>▼ 悪意の種類の内訳（指定がない分は「悪意なし」になります）</h6>")
                with gr.Row():
                    num_hasty_input = gr.Number(label="早まった一般化", value=0, step=1, minimum=0)
                    num_ad_hominem_input = gr.Number(label="対人攻撃", value=0, step=1, minimum=0)
                    num_straw_man_input = gr.Number(label="藁人形論法", value=0, step=1, minimum=0)
            def toggle_malice_details(choice):
                return gr.update(visible=choice == "悪意あり")
            malice_choice_radio.change(fn=toggle_malice_details, inputs= malice_choice_radio, outputs=malice_details_group)

            generate_btn = gr.Button("① 統合プロセス(V7)を実行し、CSVを出力", variant="primary")

        with gr.Column(scale=2):
            export_status_output = gr.Markdown(label="処理ステータス")
            opinions_output = gr.Textbox(label="生成された意見（プレビュー）", lines=15, interactive=False)
            dataframe_preview = gr.Dataframe(label="生成結果プレビュー（CSV内容）", wrap=True, row_count=3)
            csv_output_file = gr.File(label="ダウンロード", interactive=False)

    gr.Examples(
        [["AIと人間の共存", 3, 1, 1, 1, "悪意なし", 0, 0, 0]],
        inputs=[theme_input, num_outputs_input, num_pro_input, num_neutral_input, num_con_input,
                malice_choice_radio, num_hasty_input, num_ad_hominem_input, num_straw_man_input]
    )

    # ▼▼▼ generate_btn.click の inputs を修正 ▼▼▼
    generate_btn.click(
        fn=generate_opinions_only,
        inputs=[
            theme_input, num_outputs_input, num_pro_input, num_neutral_input, num_con_input,
            malice_choice_radio, num_hasty_input, num_ad_hominem_input, num_straw_man_input,
            unified_api_key_input,
            model_input
        ],
        outputs=[export_status_output, opinions_output, dataframe_preview, csv_output_file]
    )
    # ▲▲▲ 修正ここまで ▲▲▲

# Gradioの起動
print("Gradioインターフェース(V7 - 1件選定)を起動します...")
demo.launch(debug=True)

MMR用のSentenceTransformerモデルをロード中...
モデルのロード完了。
Gradioインターフェース(V7 - 1件選定)を起動します...
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://055587cc0dc6a91fff.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
